In [1]:
import csv
import glob
import math

import cv2
import dlib
import numpy as np

from collections import OrderedDict

In [2]:
SOURCE_PATH = "dgw/input"
RESULT_PATH = "dgw/preprocessed"

FACE_DETECTOR_MODEL = "pre_trained_models/mmod_human_face_detector.dat"
PROCESSING_DEVICE = "cpu"

In [3]:
class CustomPoint:
    def __init__(self, X, Y):
        self.x = X
        self.y = Y

class CustomRectangle:
    def __init__(self, topLeft, bottomRight):
        self.topLeft = topLeft
        self.bottomRight = bottomRight
        
        self.height = abs(topLeft.y - bottomRight.y)
        self.width = abs(bottomRight.x - topLeft.x)
    
    def get_area(self):
        area = self.height * self.width
        return abs(area)
    

In [4]:
def boundary_check(val, min_threshold=None, max_threshold=None):
    if min_threshold is not None and val < min_threshold:
        val = min_threshold
    elif max_threshold is not None and val > max_threshold:
        val = max_threshold
    return val

In [5]:
def get_facial_landmarks(img):
    import face_alignment
    face_detector = face_alignment.FaceAlignment(
        face_alignment.LandmarksType._2D,
        flip_input=False,
        device=PROCESSING_DEVICE
    )

    facial_landmarks = face_detector.get_landmarks(img)
    return facial_landmarks[0]

In [6]:
def get_driver_face(img):
    upsample_num = 1

    hog_face_detector = dlib.get_frontal_face_detector()
    cnn_face_detector = dlib.cnn_face_detection_model_v1(FACE_DETECTOR_MODEL)
    
    detected_faces = hog_face_detector(img, 1)
    is_using_cnn = False
    if not detected_faces:
        detected_faces = cnn_face_detector(img, upsample_num)
        is_using_cnn = True

    if not detected_faces:
        raise Exception("No faces found.")
    
    main_face = None
    main_face_area = 0

#     print("FACE DETECTED ", len(detected_faces))
    for i in range(len(detected_faces)):
        current_face = detected_faces[i]
        if is_using_cnn:
            current_face = detected_faces[i].rect
        
        top_left = CustomPoint(
            boundary_check(current_face.left(), 0, img.shape[1]), 
            boundary_check(current_face.top(), 0, img.shape[0])
        )
        bottom_right = CustomPoint(
            boundary_check(current_face.right(), 0, img.shape[1]), 
            boundary_check(current_face.bottom(), 0, img.shape[0])
        )
    
        current_face_area = CustomRectangle(top_left, bottom_right).get_area()
        
        if current_face_area > main_face_area:
            main_face = current_face
            main_face_area = current_face_area

    return img[
        boundary_check(main_face.top(), 0, img.shape[0]):boundary_check(main_face.bottom(), 0, img.shape[1]), 
        boundary_check(main_face.left(), 0, img.shape[0]):boundary_check(main_face.right(), 0, img.shape[1])]

In [7]:
def get_head_pose(img, landmarks):
    size = img.shape

    image_points = np.array([
                            landmarks[33],     # Nose tip
                            landmarks[8],      # Chin
                            landmarks[36],     # Left eye left corner
                            landmarks[45],     # Right eye right corne
                            landmarks[48],     # Left Mouth corner
                            landmarks[54]      # Right mouth corner
                        ], dtype=float)
                        
    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-165.0, 170.0, -135.0),     # Left eye left corner
                            (165.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                         
                        ], dtype=float)

    # Camera internals
    center = (size[1]/2, size[0]/2)
    focal_length = center[0] / np.tan(60/2 * np.pi / 180)
    camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = float
                         )

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

    
    axis = np.float32([[500,0,0], 
                          [0,500,0], 
                          [0,0,500]])
                          
    imgpts, jac = cv2.projectPoints(axis, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    modelpts, jac2 = cv2.projectPoints(model_points, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    rvec_matrix = cv2.Rodrigues(rotation_vector)[0]

    proj_matrix = np.hstack((rvec_matrix, translation_vector))
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6] 

    
    pitch, yaw, roll = [math.radians(_) for _ in eulerAngles]


    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))

    return imgpts, modelpts, str(int(roll)), str(int(pitch)), str(int(yaw))


In [8]:
def get_extra_padding(landmarks):
    return np.linalg.norm(landmarks[38] - landmarks[20])

def get_driver_eyes(img, landmarks):
    
    FACIAL_LANDMARKS_IDXS = OrderedDict([
        ("mouth", (48, 68)),
        ("right_eyebrow", (17, 22)),
        ("left_eyebrow", (22, 27)),
        ("right_eye", (36, 42)),
        ("left_eye", (42, 48)),
        ("nose", (27, 35)),
        ("jaw", (0, 17))
    ])
    
    left_eye = landmarks[FACIAL_LANDMARKS_IDXS["left_eye"][0]:FACIAL_LANDMARKS_IDXS["left_eye"][1]]
    right_eye = landmarks[FACIAL_LANDMARKS_IDXS["right_eye"][0]:FACIAL_LANDMARKS_IDXS["right_eye"][1]]
    left_eyebrow = landmarks[FACIAL_LANDMARKS_IDXS["left_eyebrow"][0]:FACIAL_LANDMARKS_IDXS["left_eyebrow"][1]]
    right_eyebrow = landmarks[FACIAL_LANDMARKS_IDXS["right_eyebrow"][0]:FACIAL_LANDMARKS_IDXS["right_eyebrow"][1]]
    
    
    eyes = np.concatenate((left_eye, right_eye, left_eyebrow, right_eyebrow))
    top_left_x = int(min(eyes, key = lambda t: t[0])[0] - get_extra_padding(landmarks))
    top_left_y = int(min(eyes, key = lambda t: t[1])[1])
    
    bottom_right_x = int(max(eyes, key = lambda t: t[0])[0] + get_extra_padding(landmarks))
    bottom_right_y = int(max(eyes, key = lambda t: t[1])[1] + get_extra_padding(landmarks))
    
#     print([
#         boundary_check(top_left_x, 0, img.shape[0]), boundary_check(bottom_right_x, 0, img.shape[0]), 
#         boundary_check(top_left_y, 0, img.shape[1]), boundary_check(bottom_right_y, 0, img.shape[1])])

    return img[
        boundary_check(top_left_y, 0, img.shape[1]):boundary_check(bottom_right_y, 0, img.shape[1]),
        boundary_check(top_left_x, 0, img.shape[0]):boundary_check(bottom_right_x, 0, img.shape[0])]

In [10]:
for file_path in glob.glob("{src}/[0-9]/*.png".format(src=SOURCE_PATH)):
    path_info = file_path.split("/")
    IMAGE_NAME = path_info[-1].replace(".png", "")
    ZONE_NAME = path_info[-2]
    
    img = dlib.load_rgb_image(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
   # Extract the face driver's face using dlib
    try:
        face_img = get_driver_face(img)
    except Exception as e:
        print("Unable to detect faces at {}".format(file_path))
    
    # Extract 2D landmarks using face_alignment
    try:
        landmarks = get_facial_landmarks(face_img)
    except Exception as e:
        print("Unable to detect landmarks at {}".format(file_path))
    
    # Extract driver eye's ROI
    try:
        eye_image = get_driver_eyes(face_img, landmarks)
    except Exception as e:
        print("Unable to get driver eye at {}".format(file_path))
    
    # Extract head pose using solvePnP
    try:
        imgpts, _, roll, pitch, yaw = get_head_pose(face_img, landmarks)
    except:
        print("Unable to fetch head pose at {}".format(file_path))
    
    
    op_path_face_cropped = "{dir}/{zone}/cropped_{img}.png".format(dir=RESULT_PATH, zone=ZONE_NAME, img=IMAGE_NAME)
    cv2.imwrite(op_path_face_cropped, face_img)

    op_path_eye_cropped = "{dir}/{zone}/eye_{img}.png".format(dir=RESULT_PATH, zone=ZONE_NAME, img=IMAGE_NAME)
    cv2.imwrite(op_path_eye_cropped, eye_image)

#     # For testing the head pose     
#     cv2.line(img, tuple(landmarks[33]), tuple(imgpts[1].ravel()), (0,255,0), 3) #GREEN
#     cv2.line(img, tuple(landmarks[33]), tuple(imgpts[0].ravel()), (255,0,), 3) #BLUE
#     cv2.line(img, tuple(landmarks[33]), tuple(imgpts[2].ravel()), (0,0,255), 3) #RED
#     op_path_head_pose = "{dir}/{zone}/headpose_{img}.png".format(dir=RESULT_PATH, zone=ZONE_NAME, img=IMAGE_NAME)
#     cv2.imwrite(op_path_head_pose, img)    
    
    feature_list = list()
    # index 0: zone name
    feature_list.append(ZONE_NAME)
    # index 1: image name
    feature_list.append(IMAGE_NAME)
    
    flat_landmarks = [item for sublist in landmarks for item in sublist]
    # index 2-137: landmarks
    feature_list.extend(flat_landmarks)
    
    # index 138-140: headpose
    feature_list.extend([roll, pitch, yaw])
    
    # index 141: face_size
    feature_list.append(face_img.shape[0])
    
    if (len(feature_list) != 142):
        print("Unable to fetch all the features for image {} in zone {}.".format(
            IMAGE_NAME, ZONE_NAME))
    else:
        feature_file = "{dir}/{zone}/feature.csv".format(dir=RESULT_PATH, zone=ZONE_NAME)
        with open(feature_file, 'a+', newline='') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=' ')
            spamwriter.writerow(feature_list)
